###Carregamento do modelo

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-32B-Instruct-AWQ"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/homeLocal/sofia/venv_awq/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/homeLocal/sofia/venv_awq/lib/python3.12/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
We suggest you to set `torch_dtype=torch.float16` for better efficiency on CUDA/XPU with AWQ.
Fetching 5 files: 100%|██████████████████████████████████████████████████████████████████| 5/5 [27:17<00:00, 327.60s/it]
/homeLocal/sofia/venv_awq/lib/python3.12/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch

###Prática 1

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code1-1.py", "Solucao/solucao-code1.py", "lint/code1-1-lint.pre", "ast/code1-1-ast.pre", "Esqueleto/esqueleto-code1.py", "Saida/saida1.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code1.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code1.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p02.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code1.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code1-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code1-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code1-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida1.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para calcular o período em que o estudante se encontra no curso, considere que cada ano tem 2 períodos letivos;
    - O programa deve informar a idade provável pois não tem informação sobre o mês de nascimento.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de um espaço para uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

- **Nome**: O aluno preencheu o campo com "Ana Maria".
- **Matrícula**: O aluno preencheu o campo com "245637".
- **Data**: O aluno preencheu o campo com "10/09/2025".
- **Descrição**: O aluno deixou o campo com o texto padrão "(substitua essa linha por um comentário dizendo o que o programa faz)" sem alterá-lo.

Portanto, o aluno preencheu os campos de nome, matrícula e data, mas não adicionou uma descrição do que o programa faz.

### Boas Práticas

O pylint apontou algumas observações importantes sobre o código do aluno. Vamos analisar essas observações:

- **Espaço após o sinal de igual**: O aluno usou parênteses desnecessários após o sinal de igual (`=`) em algumas atribuições. Isso não afeta a funcionalidade do código, mas é consid

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code1-2.py", "Solucao/solucao-code1.py", "lint/code1-2-lint.pre", "ast/code1-2-ast.pre", "Esqueleto/esqueleto-code1.py", "Saida/saida1.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code1.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code1.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p02.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code1.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code1-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code1-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code1-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida1.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para calcular o período em que o estudante se encontra no curso, considere que cada ano tem 2 períodos letivos;
    - O programa deve informar a idade provável pois não tem informação sobre o mês de nascimento.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

**Preenchimento dos campos obrigatórios:**
- **Nome:** O aluno preencheu o campo com "Bruna Souza".
- **Matrícula:** O aluno preencheu o campo com "144657".
- **Data:** O aluno preencheu o campo com "27/08/2025".

**Descrição do programa:**
- O aluno adicionou uma breve descrição do que o programa faz: "O programa identifica a idade e em qual período o aluno entrou na UFV." Esta descrição é adequada e clara.

### Boas Práticas

**Observações do pylint:**
- **Linhas muito longas:** A linha 17 do código é muito longa (171 caracteres contra o limite de 100). É recomendável dividir essa linha em várias para melhorar a legibilidade.
- **Nomes de variáveis e constantes:** A constante `anoAtual` deve ser escrita em letras maiúsculas (`ANO_ATUAL`) para seguir as convenções de nomenclatura do Python. Além disso, o nome da variável `anoInicioDoCurso` pode ser melhorado para `ano_inicio_do_curso`, seguindo o estilo `snake_case`.

### Lógica e Concordância com o Roteiro

**Cálculo d

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code1-3.py", "Solucao/solucao-code1.py", "lint/code1-3-lint.pre", "ast/code1-3-ast.pre", "Esqueleto/esqueleto-code1.py", "Saida/saida1.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code1.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code1.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p02.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code1.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code1-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code1-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code1-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida1.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para calcular o período em que o estudante se encontra no curso, considere que cada ano tem 2 períodos letivos;
    - O programa deve informar a idade provável pois não tem informação sobre o mês de nascimento.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

No código base fornecido pelo professor, havia campos para que o aluno pudesse preencher o nome, matrícula e data, além de uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

- **Nome do aluno:** O aluno preencheu com "Bárbara Silveira". Isso está correto.
- **Matrícula:** O aluno preencheu com "144659". Isso está correto.
- **Data:** O aluno preencheu com "27/08/2025". Isso está correto.
- **Descrição do programa:** O aluno escreveu "O programa fala idade, período e qual período de ingresso do aluno na UFV." Esta descrição está adequada, embora possa ser mais detalhada.

### Boas Práticas

O pylint apontou vários problemas no código do aluno. Vamos analisar os principais:

- **Linhas muito longas:** O pylint indicou que a linha 17 do código é muito longa (113 caracteres contra o limite de 100). É importante manter as linhas de código curtas para facilitar a leitura e manutenção.
- **Espaços em branco desnecessários:

###Prática 2

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code2-1.py", "Solucao/solucao-code2.py", "lint/code2-1-lint.pre", "ast/code2-1-ast.pre", "Esqueleto/esqueleto-code2.py", "Saida/saida2.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code2.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code2.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p03.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code2.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code2-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code2-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code2-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida2.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para qualquer valor diferente de 1, a escolha é a opção 2.
    - A tarefa do aluno é incluir um único if-else no código.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de um espaço para uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

**Campos Preenchidos pelo Aluno:**
- **Nome do aluno:** Armando Batista
- **Matrícula:** 528694
- **Data:** 03/09/2025
- **Breve descrição:** O aluno não adicionou uma breve descrição do que o programa faz. O campo permaneceu como `(breve comentário dizendo o que o programa faz)`.

**Conclusão:**
O aluno preencheu os campos de nome, matrícula e data corretamente, mas esqueceu de adicionar uma breve descrição do que o programa faz.

#### Boas Práticas

O pylint apontou algumas observações importantes sobre o código do aluno. Vamos analisar essas observações:

1. **Parênteses desnecessários após `if`:**
   - **Observação:** O aluno usou parênteses após a palavra-chave `if`, o que é desnecessário em Python.

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code2-2.py", "Solucao/solucao-code2.py", "lint/code2-2-lint.pre", "ast/code2-2-ast.pre", "Esqueleto/esqueleto-code2.py", "Saida/saida2.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code2.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code2.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p03.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code2.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code2-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code2-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code2-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida2.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para qualquer valor diferente de 1, a escolha é a opção 2.
    - A tarefa do aluno é incluir um único if-else no código.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de um espaço para uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

**Campos Preenchidos pelo Aluno:**
- **Nome do aluno:** Bianca Andrade
- **Matrícula:** 101457
- **Data:** 03/09/2025
- **Breve descrição:** Faltou preencher a descrição do que o programa faz.

**Observações:**
- O aluno preencheu corretamente os campos de nome, matrícula e data.
- No entanto, o campo de descrição do que o programa faz foi deixado em branco. É importante que o aluno escreva uma breve descrição para que outros possam entender rapidamente o propósito do programa.

#### Boas Práticas

O pylint apontou algumas observações importantes sobre o código do aluno. Vamos analisar essas observações:

- **Trailing whitespace (trailing-whitespace):** Há espaços em branco desnecessários no final da lin

###Prática 3

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code3-1.py", "Solucao/solucao-code3.py", "lint/code3-1-lint.pre", "ast/code3-1-ast.pre", "Esqueleto/esqueleto-code3.py", "Saida/saida3.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code3.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code3.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p04.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code3.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code3-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code3-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code3-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida3.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para crianças com menos de 4 anos e idosos com 65 anos ou mais, o valor do ingresso é 0 reais e é sem acesso.
    - Para crianças de 4 a 12 anos, o valor do ingresso deve ser 75 reais, de 13 a 17 deve ser 85 reais e de 18 ou mais, 115 reais.
    - A altura só deve ser solicitada quando é necessária para definir o tipo de ingresso, ou seja, apenas nos casos do tópico anterior, sendo acesso restrito para menos de 1.50 de altura (aplicando desconto de 10% no valor do ingresso) e acesso total para altura maior ou igual a 1.50.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno pudesse preencher o nome, matrícula e data, além de uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

- **Nome do aluno:** O aluno preencheu com "Bianca Andrade".
- **Matrícula:** O aluno preencheu com "101457".
- **Data:** O aluno preencheu com "10/09/2025".
- **Breve descrição:** O aluno escreveu "O programa gerencia o acesso ao parque de acordo com idade e altura".

Portanto, todos os campos foram preenchidos conforme solicitado.

#### Boas Práticas

O pylint apontou vários problemas no código do aluno. Vamos analisar os principais:

- **Indentação incorreta:** O pylint indicou que a indentação estava incorreta em várias partes do código. Em Python, a indentação é crucial para definir blocos de código. O aluno usou 6 espaços em alguns lugares, enquanto o padrão é 4 espaços.
  
- **Uso de nomes de variáveis:** 

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code3-2.py", "Solucao/solucao-code3.py", "lint/code3-2-lint.pre", "ast/code3-2-ast.pre", "Esqueleto/esqueleto-code3.py", "Saida/saida3.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code3.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code3.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p04.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code3.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code3-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code3-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code3-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida3.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para crianças com menos de 4 anos e idosos com 65 anos ou mais, o valor do ingresso é 0 reais e é sem acesso.
    - Para crianças de 4 a 12 anos, o valor do ingresso deve ser 75 reais, de 13 a 17 deve ser 85 reais e de 18 ou mais, 115 reais.
    - A altura só deve ser solicitada quando é necessária para definir o tipo de ingresso, ou seja, apenas nos casos do tópico anterior, sendo acesso restrito para menos de 1.50 de altura (aplicando desconto de 10% no valor do ingresso) e acesso total para altura maior ou igual a 1.50.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

No código base fornecido pelo professor, havia campos para que o aluno pudesse preencher o nome, matrícula e data, além de uma breve descrição do que o programa faz. No código do aluno Caio Pires, esses campos foram preenchidos conforme solicitado:

- **Nome do aluno:** Caio Pires
- **Matrícula:** 115747
- **Data:** 10/09/2025

Além disso, o aluno adicionou uma breve descrição do que o programa faz: "Dependendo da idade e altura é definido o acesso aos brinquedos". Esta descrição, embora curta, atende ao requisito de fornecer uma breve explicação do propósito do programa.

### Boas Práticas

O aluno recebeu algumas observações do pylint que podem ser resumidas e explicadas em português:

- **Uso de nomes de variáveis:** O pylint apontou que as variáveis `tipo` e `valor` não seguem o estilo de nomenclatura recomendado para constantes, que seria `UPPER_CASE`. No entanto, neste contexto, essas variáveis não são constantes, mas sim variáveis comuns, então o estilo adequado s

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code3-3.py", "Solucao/solucao-code3.py", "lint/code3-3-lint.pre", "ast/code3-3-ast.pre", "Esqueleto/esqueleto-code3.py", "Saida/saida3.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code3.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code3.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p04.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code3.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code3-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code3-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code3-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida3.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para crianças com menos de 4 anos e idosos com 65 anos ou mais, o valor do ingresso é 0 reais e é sem acesso.
    - Para crianças de 4 a 12 anos, o valor do ingresso deve ser 75 reais, de 13 a 17 deve ser 85 reais e de 18 ou mais, 115 reais.
    - A altura só deve ser solicitada quando é necessária para definir o tipo de ingresso, ou seja, apenas nos casos do tópico anterior, sendo acesso restrito para menos de 1.50 de altura (aplicando desconto de 10% no valor do ingresso) e acesso total para altura maior ou igual a 1.50.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno pudesse preencher o nome, matrícula e data, além de uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

**Campos preenchidos pelo aluno:**
- **Nome do aluno:** Miguel Pires
- **Matrícula:** 115749
- **Data:** 10/09/2025
- **Descrição:** A idade e altura da pessoa define o valor e tipo de ingresso

**Observações:**
- O aluno preencheu todos os campos solicitados no cabeçalho do código.
- A descrição fornecida é bastante curta, mas atende ao pedido de descrever o que o programa faz.

#### Boas Práticas

O pylint apontou algumas observações sobre o código do aluno. Vamos analisar as principais:

- **Trailing whitespace (espaços em branco no final da linha):** O aluno deixou um espaço em branco no final da linha 9. É importante manter o código limpo e sem espaços desnecessários no final das linhas.
- **Nomenclatura de 

###Prática 4

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code4-1.py", "Solucao/solucao-code4.py", "lint/code4-1-lint.pre", "ast/code4-1-ast.pre", "Esqueleto/esqueleto-code4.py", "Saida/saida4.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code4.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code4.py']}

Para resolver a atividade, o professor fornece o seguinte código base para o aluno completar:

{codes['Esqueleto/esqueleto-code4.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code4-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code4-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code4-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida4.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se a idade do aluno for >= a 16 anos, ele pode participar e não precisa solicitar tempo de escolinha.
    - Em caso negativo, se o tempo de escolhinha for >= 5 anos, ele pode participar.
    - Em caso negativo, se a soma do tempo de escolinha e idade for >= 18 anos, ele pode participar.
    - Caso nenhuma das condições seja atendida, o aluno não pode participar.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de um espaço para uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

**Campos Preenchidos pelo Aluno:**
- **Nome:** Cássio Pereira
- **Matrícula:** 125478
- **Data:** 17/09/25
- **Descrição:** O programa define condições para solucionar um problema.

**Observações:**
- O aluno preencheu todos os campos solicitados (nome, matrícula e data).
- A descrição do programa é bastante genérica e não especifica claramente o que o programa faz. Uma descrição mais detalhada seria útil, por exemplo: "O programa verifica se um aluno tem permissão para participar de um campeonato de futebol com base em sua idade e tempo de escolinha."

#### Boas Práticas

**Observações do Pylint:**
- **Trailing whitespace (trailing-whitespace):** Há espaços em branco desnecessários no final de algumas l

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code4-2.py", "Solucao/solucao-code4.py", "lint/code4-2-lint.pre", "ast/code4-2-ast.pre", "Esqueleto/esqueleto-code4.py", "Saida/saida4.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code4.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code4.py']}

Para resolver a atividade, o professor fornece o seguinte código base para o aluno completar:

{codes['Esqueleto/esqueleto-code4.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code4-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code4-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code4-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida4.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se a idade do aluno for >= a 16 anos, ele pode participar e não precisa solicitar tempo de escolinha.
    - Em caso negativo, se o tempo de escolhinha for >= 5 anos, ele pode participar.
    - Em caso negativo, se a soma do tempo de escolinha e idade for >= 18 anos, ele pode participar.
    - Caso nenhuma das condições seja atendida, o aluno não pode participar.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

No código base fornecido pelo professor, havia campos para que o aluno pudesse preencher informações como nome, matrícula e data, além de um espaço para uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

**Campos preenchidos pelo aluno:**
- **Nome:** Gabriel Fernandes
- **Matrícula:** 123456
- **Data:** 18/08/2025
- **Descrição:** "O código recebe a idade e verifica se os alunos podem ou não participar do campeonato"

**Observações:**
- O aluno preencheu todos os campos solicitados (nome, matrícula e data).
- O aluno também adicionou uma breve descrição do que o programa faz, embora ela possa ser mais detalhada para refletir completamente o funcionamento do programa.

### Boas Práticas

O pylint apontou vários problemas no código do aluno. Vamos analisar esses problemas e sugerir melhorias.

**Problemas identificados:**
- **Indentação incorreta:** O pylint apontou que a indentação está incorreta em várias linhas. Py

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code4-3.py", "Solucao/solucao-code4.py", "lint/code4-3-lint.pre", "ast/code4-3-ast.pre", "Esqueleto/esqueleto-code4.py", "Saida/saida4.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code4.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code4.py']}

Para resolver a atividade, o professor fornece o seguinte código base para o aluno completar:

{codes['Esqueleto/esqueleto-code4.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code4-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code4-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code4-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida4.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se a idade do aluno for >= a 16 anos, ele pode participar e não precisa solicitar tempo de escolinha.
    - Em caso negativo, se o tempo de escolhinha for >= 5 anos, ele pode participar.
    - Em caso negativo, se a soma do tempo de escolinha e idade for >= 18 anos, ele pode participar.
    - Caso nenhuma das condições seja atendida, o aluno não pode participar.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de um espaço para uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

**Campos Preenchidos pelo Aluno:**
- **Nome do aluno:** Mara Silva
- **Matrícula:** 125478
- **Data:** 10/09/2025
- **Descrição:** O campo de descrição permaneceu vazio.

**Observações:**
- O aluno preencheu corretamente os campos de nome, matrícula e data.
- No entanto, o campo de descrição não foi preenchido, o que é uma boa prática para documentar o propósito do programa.

#### Boas Práticas

O pylint apontou algumas observações importantes sobre o código do aluno. Vamos analisar essas observações:

- **Considerar o uso de f-string:** O pylint sugere o uso de f-string em vez de formatação de strings regulares. Isso torna o código mais legível e eficiente. Por exemplo, em vez de usar `'%s, você pode jo

###Prática 5

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code5-1.py", "Solucao/solucao-code5.py", "lint/code5-1-lint.pre", "ast/code5-1-ast.pre", "Esqueleto/esqueleto-code5.py",  "Esqueleto/esqueleto-code5-2.py", "Saida/saida5.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code5.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code5.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p06.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code5.py']}

E fornece também o código repetitivo.py, que mostra como usar o comando while:

{codes['Esqueleto/esqueleto-code5-2.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code5-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code5-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code5-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida5.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O programa deve executar pelo menos uma vez, perguntando, a cada vez, se o usuário deseja repetir a execução.
    - A lógica do repetitivo.py pode ser aplicada pelo aluno, assim como ele pode usar outra, desde que cumpra a tarefa
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de um espaço para uma breve descrição do que o programa faz. Vamos analisar se esses campos foram preenchidos corretamente no código do aluno.

- **Nome do aluno:** O aluno preencheu com "Fabricio Kenji".
- **Matrícula:** O aluno preencheu com "142587".
- **Data:** O aluno preencheu com "24-09-2025".
- **Descrição do programa:** O aluno adicionou a descrição "Este programa calcula as raízes de uma equação quantas vezes o usuário desejar".

Portanto, todos os campos solicitados foram preenchidos corretamente.

### Boas Práticas

O pylint apontou algumas observações sobre o código do aluno. Vamos analisar as observações relevantes:

- **Trailing newlines (linhas em branco no final do arquivo):** É uma boa prática garantir que o arquivo de código não termine com linhas em branco desnecessárias. Isso ajuda a manter a limpeza do código e evita problemas em sistema

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code5-2.py", "Solucao/solucao-code5.py", "lint/code5-2-lint.pre", "ast/code5-2-ast.pre", "Esqueleto/esqueleto-code5.py",  "Esqueleto/esqueleto-code5-2.py", "Saida/saida5.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code5.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code5.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p06.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code5.py']}

E fornece também o código repetitivo.py, que mostra como usar o comando while:

{codes['Esqueleto/esqueleto-code5-2.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code5-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code5-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code5-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida5.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O programa deve executar pelo menos uma vez, perguntando, a cada vez, se o usuário deseja repetir a execução.
    - A lógica do repetitivo.py pode ser aplicada pelo aluno, assim como ele pode usar outra, desde que cumpra a tarefa
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de um espaço para uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

- **Nome do aluno:** O aluno preencheu o campo com "Kaiki Lopes".
- **Matrícula:** O aluno preencheu o campo com "142563".
- **Data:** O aluno preencheu o campo com "24/09/25". Note que a data parece estar incorreta, pois o dia 25 de setembro não existe. Provavelmente era para ser "24/09/2025".
- **Descrição do programa:** O aluno não preencheu o campo para a descrição do programa. O campo ainda contém o texto padrão fornecido pelo professor.

### Boas Práticas

O pylint apontou algumas observações importantes sobre o código do aluno. Vamos analisar essas observações:

- **Trailing whitespace (espaços em branco no final das linhas):** O aluno deixou espaços em branco no final de algumas linhas (linhas 6, 20 e 50). Isso pode causar probl

###Prática 6

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code6-1.py", "Solucao/solucao-code6.py", "lint/code6-1-lint.pre", "ast/code6-1-ast.pre", "Esqueleto/esqueleto-code6.py", "Saida/saida6.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code6.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code6.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p07.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code6.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code6-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code6-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code6-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida6.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se o palpite do usuário estiver dentro do intervalo, deve ser considerado como palpite válido e incrementado no contador.
    - Adicionalmente, se o palpite estiver errado, o código deve retornar se o número sorteado é maior ou menor do que o palpite e definir novo intervalo com base no palpite.
    - A semente só deve ser inicializada uma vez e o intervalo deve voltar a ser [1,100] caso o usuário acerte o número sorteado e decida jogar novamente.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

**Preenchimento dos Campos:**
- **Nome:** O aluno preencheu o campo nome com "Lucas Marques".
- **Matrícula:** O aluno preencheu o campo matrícula com "121475".
- **Data:** O aluno preencheu o campo data com "29/09/2025".

**Descrição do Programa:**
- O aluno não adicionou uma breve descrição do que o programa faz. O campo onde deveria ser escrita a descrição permanece vazio.

### Boas Práticas

**Erros Apontados pelo Pylint:**

1. **Espaços em Branco Inúteis (Trailing Whitespace):**
   - O aluno deixou espaços em branco desnecessários no final de várias linhas. Isso pode causar problemas de formatação e é considerado uma má prática.

2. **Indentação Incorreta:**
   - O aluno usou uma indentação inconsistente, utilizando 9 espaços em alguns lugares e 15 em outros, enquanto o padrão geralmente é 4 espaços por nível de indentação. Isso pode levar a erros de execução e dificultar a leitura do código.

3. **Nomes de Variáveis:**
   - As variáveis `linf`, `lsup` e `cont` não 

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code6-2.py", "Solucao/solucao-code6.py", "lint/code6-2-lint.pre", "ast/code6-2-ast.pre", "Esqueleto/esqueleto-code6.py", "Saida/saida6.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code6.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code6.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p07.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code6.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code6-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code6-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code6-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida6.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se o palpite do usuário estiver dentro do intervalo, deve ser considerado como palpite válido e incrementado no contador.
    - Adicionalmente, se o palpite estiver errado, o código deve retornar se o número sorteado é maior ou menor do que o palpite e definir novo intervalo com base no palpite.
    - A semente só deve ser inicializada uma vez e o intervalo deve voltar a ser [1,100] caso o usuário acerte o número sorteado e decida jogar novamente.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

**Preenchimento dos Campos:**
- **Nome:** O aluno preencheu o campo nome com "Maria Souza".
- **Matrícula:** O aluno preencheu o campo matrícula com "125784".
- **Data:** O aluno preencheu o campo data com "31/09/25".

**Descrição do Programa:**
- O aluno adicionou uma breve descrição do que o programa faz: "Um número é sorteado e o operador deve tentar adivinhar como uma brincadeira de criança". Esta descrição é suficiente e clara.

### Boas Práticas

**Observações do Pylint:**
- **Trailing Whitespace:** Existem vários espaços em branco desnecessários no final das linhas. É importante remover esses espaços para manter o código limpo e evitar erros futuros.
- **Nome de Variáveis:** As variáveis `linf`, `lsup` e `palpites` não seguem a convenção de nomes em Python, que recomenda nomes em minúsculas com underscores para constantes e variáveis. Embora isso não seja um erro crítico, é uma boa prática seguir essa convenção para melhorar a legibilidade do código.
- **Estrutura

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code6-3.py", "Solucao/solucao-code6.py", "lint/code6-3-lint.pre", "ast/code6-3-ast.pre", "Esqueleto/esqueleto-code6.py", "Saida/saida6.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code6.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code6.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p07.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code6.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code6-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code6-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code6-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida6.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se o palpite do usuário estiver dentro do intervalo, deve ser considerado como palpite válido e incrementado no contador.
    - Adicionalmente, se o palpite estiver errado, o código deve retornar se o número sorteado é maior ou menor do que o palpite e definir novo intervalo com base no palpite.
    - A semente só deve ser inicializada uma vez e o intervalo deve voltar a ser [1,100] caso o usuário acerte o número sorteado e decida jogar novamente.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

**Preenchimento dos Campos:**
- **Nome:** O aluno preencheu o campo nome com "Maria Roberta".
- **Matrícula:** O aluno preencheu o campo matrícula com "125786".
- **Data:** O aluno preencheu o campo data com "31/09/25".

**Descrição do Programa:**
- O aluno adicionou uma breve descrição do que o programa faz: "O usuário deve adivinhar o número sorteado". Esta descrição é bastante simples e direta, mas atende ao requisito de fornecer uma breve explicação do propósito do programa.

### Boas Práticas

**Observações do Pylint:**
- **Trailing Whitespace:** Existem vários espaços em branco desnecessários no final das linhas. É importante remover esses espaços para manter o código limpo e evitar erros futuros.
- **Invalid Name:** As variáveis `linf`, `lsup` e `palpites` não seguem o estilo de nomenclatura recomendado para variáveis em Python, que é `snake_case`. É recomendável renomear essas variáveis para `limite_inferior`, `limite_superior` e `numero_palpites`, respectivament

###Prática 7

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code7-1.py", "Solucao/solucao-code7.py", "lint/code7-1-lint.pre", "ast/code7-1-ast.pre", "Esqueleto/esqueleto-code7.py", "Saida/saida7.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code7.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code7.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p08.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code7.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code7-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code7-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code7-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida7.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Como primeira tarefa, o programa deve ler uma semente inteira e, se ela for negativa, deve pedir o valor da semente novamente até que seja digitado um valor >=0.
    - Como segunda tarefa, ele deve percorrer o arranjo e contar quantos valores são maiores do que a média inteira já implementada pelo professor na solução
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de um espaço para uma breve descrição do que o programa faz. Vamos analisar se esses campos foram preenchidos corretamente:

- **Nome do aluno:** Maria Souza
- **Matrícula:** 125784
- **Data:** 08/10/25
- **Descrição:** O campo foi substituído por um comentário genérico, mas não há uma descrição específica do que o programa faz. O aluno poderia ter colocado algo como "Este programa gera um arranjo de 25 números inteiros no intervalo [0, 9], calcula a média inteira e conta quantos valores são maiores que essa média."

Portanto, o aluno preencheu os campos de nome, matrícula e data, mas não adicionou uma descrição adequada do programa.

#### Boas Práticas

O pylint apontou algumas observações importantes que o aluno pode considerar para melhorar o código:

- **Uso de `enumerate`:** Em dois pontos do código, o aluno está usando `

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code7-2.py", "Solucao/solucao-code7.py", "lint/code7-2-lint.pre", "ast/code7-2-ast.pre", "Esqueleto/esqueleto-code7.py", "Saida/saida7.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code7.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code7.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p08.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code7.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code7-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code7-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code7-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida7.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Como primeira tarefa, o programa deve ler uma semente inteira e, se ela for negativa, deve pedir o valor da semente novamente até que seja digitado um valor >=0.
    - Como segunda tarefa, ele deve percorrer o arranjo e contar quantos valores são maiores do que a média inteira já implementada pelo professor na solução
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

**Nome, Matrícula e Data:**
- **Nome:** O aluno preencheu o campo com "Mirian Silva".
- **Matrícula:** O aluno preencheu o campo com "99578".
- **Data:** O aluno preencheu o campo com "08/10/2025".

**Descrição do Programa:**
- O aluno adicionou uma breve descrição do que o programa faz: "O código verifica se o valor digitado é maior que zero e vê a média dos 25 números do arranjo... e depois imprime os números maiores que a média."

Esses campos foram preenchidos corretamente e a descrição do programa está adequada.

#### Boas Práticas

**Observações do Pylint:**

- **Uso de `enumerate`:** É sugerido o uso de `enumerate` em vez de `range(len(...))`. Isso torna o código mais legível e eficiente.
  - Exemplo: No lugar de `for i in range(len(A))`, use `for i, valor in enumerate(A)`.

- **Nomenclatura de Variáveis:** As constantes `linf` e `lsup` devem ser escritas em letras maiúsculas (`LINF` e `LSUP`) para indicar que são constantes.
  - E

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code7-3.py", "Solucao/solucao-code7.py", "lint/code7-3-lint.pre", "ast/code7-3-ast.pre", "Esqueleto/esqueleto-code7.py", "Saida/saida7.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code7.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code7.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p08.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code7.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code7-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code7-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code7-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida7.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Como primeira tarefa, o programa deve ler uma semente inteira e, se ela for negativa, deve pedir o valor da semente novamente até que seja digitado um valor >=0.
    - Como segunda tarefa, ele deve percorrer o arranjo e contar quantos valores são maiores do que a média inteira já implementada pelo professor na solução
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de uma breve descrição do que o programa faz. Vamos analisar se esses campos foram preenchidos corretamente:

- **Nome do aluno:** Carla Márcia
- **Matrícula:** 526487
- **Data:** 10/11/2025
- **Descrição:** O programa confere quantos números são maiores do que a média dos valores de um array

**Observações:**
- O aluno preencheu corretamente os campos de nome, matrícula e data.
- A descrição do programa está adequada e descreve claramente o objetivo do programa.

#### Boas Práticas

O pylint apontou vários problemas no código do aluno. Vamos analisar esses problemas, focando apenas nas observações relevantes para o que o aluno implementou:

- **Uso de `enumerate`:** O pylint sugere o uso de `enumerate` em vez de `range(len(...))`. Isso torna o código mais legível e eficiente.
- **Nomenclatura de variáveis:** O pylint indica q

###Prática 8

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code8-1.py", "Solucao/solucao-code8.py", "lint/code8-1-lint.pre", "ast/code8-1-ast.pre", "Esqueleto/esqueleto-code8.py", "Saida/saida8.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code8.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code8.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p09.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code8.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code8-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code8-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code8-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida8.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve ordenar os elentos do arranjo A em ordem crescente sem poder usar a função sort, imprimindo esse arranjo ordenado depois.
    - Determinar e informar o valor da mediana: que é o elemento da posição do meio em caso de arranjo com tamanho ímpar ou a média aritmética dos dois valores centrais em caso de arranjo com tamanho par.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

**Nome, Matrícula e Data:**
- O aluno preencheu o campo nome (`Maria Fernanda`), matrícula (`145896`) e data (`22/10/2025`). Esses campos foram preenchidos corretamente e diferem do código base fornecido pelo professor.

**Descrição do Programa:**
- O aluno adicionou uma breve descrição do que o programa faz: `"O programa faz um arranjo de 1 a 100 ficar ordenado e exibe o valor da mediana deles."`. Esta descrição é adequada e clara.

### Boas Práticas

**Erros Apontados pelo Pylint:**
- **Erro de Indentação:** Houve um erro de indentação após a instrução `else`. O código dentro do bloco `else` precisa estar indentado corretamente. Isso é crucial para que o Python entenda que o código pertence ao bloco `else`.

### Lógica e Concordância com o Roteiro

**Ordenação do Arranjo:**
- O aluno implementou corretamente o método de seleção para ordenar o arranjo. Este método foi aplicado conforme solicitado no roteiro, sem o uso da função `sort()`.

**Determinação da Mediana:**
- 

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code8-2.py", "Solucao/solucao-code8.py", "lint/code8-2-lint.pre", "ast/code8-2-ast.pre", "Esqueleto/esqueleto-code8.py", "Saida/saida8.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code8.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code8.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p09.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code8.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code8-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code8-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code8-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida8.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve ordenar os elentos do arranjo A em ordem crescente sem poder usar a função sort, imprimindo esse arranjo ordenado depois.
    - Determinar e informar o valor da mediana: que é o elemento da posição do meio em caso de arranjo com tamanho ímpar ou a média aritmética dos dois valores centrais em caso de arranjo com tamanho par.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

No código base fornecido pelo professor, havia campos para que o aluno pudesse preencher o nome, matrícula e data, além de uma breve descrição do que o programa faz. Vamos analisar se esses campos foram preenchidos corretamente no código do aluno.

- **Nome do aluno:** O aluno preencheu com "Matheus Jorge".
- **Matrícula:** O aluno preencheu com "236587".
- **Data:** O aluno preencheu com "22/10/2025".
- **Descrição do programa:** O aluno escreveu "O programa ordena um arranjo A e calcula a mediana dos elementos". Esta descrição é adequada e concisa.

Portanto, o aluno preencheu todos os campos solicitados corretamente.

### Boas Práticas

O pylint apontou algumas observações importantes que devem ser consideradas:

- **Trailing whitespace:** Existem espaços em branco desnecessários no final de várias linhas. É importante remover esses espaços para manter o código limpo e evitar problemas futuros.
- **Nomenclatura de variáveis:** As constantes `linf` e `lsup` devem ser e

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code8-3.py", "Solucao/solucao-code8.py", "lint/code8-3-lint.pre", "ast/code8-3-ast.pre", "Esqueleto/esqueleto-code8.py", "Saida/saida8.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code8.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code8.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p09.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code8.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code8-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code8-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code8-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida8.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve ordenar os elentos do arranjo A em ordem crescente sem poder usar a função sort, imprimindo esse arranjo ordenado depois.
    - Determinar e informar o valor da mediana: que é o elemento da posição do meio em caso de arranjo com tamanho ímpar ou a média aritmética dos dois valores centrais em caso de arranjo com tamanho par.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

No código base fornecido pelo professor, havia campos para que o aluno pudesse preencher o nome, matrícula e data, além de uma breve descrição do que o programa faz. Vamos analisar se esses campos foram preenchidos corretamente no código do aluno.

- **Nome do aluno:** Maria Fernanda
- **Matrícula:** 894576
- **Data:** 28/10/2025

Esses campos foram preenchidos corretamente no código do aluno.

- **Descrição do programa:** O campo onde o aluno deveria colocar uma breve descrição do que o programa faz ainda contém o texto padrão do código base: "Substitua esta linha por um breve comentário dizendo o que o programa faz". Portanto, o aluno não preencheu essa parte.

### Boas Práticas

O pylint apontou vários problemas no código do aluno, principalmente relacionados a nomes de variáveis e espaços em branco. Vamos focar nas observações relevantes:

- **Nomes de variáveis:** As constantes `linf` e `lsup` devem ser escritas em letras maiúsculas, conforme a convenção PEP 8. Isso

###Prática 9

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code9-1.py", "Solucao/solucao-code9.py", "lint/code9-1-lint.pre", "ast/code9-1-ast.pre", "Esqueleto/esqueleto-code9.py", "Saida/saida9.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code9.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code9.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p10.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code9.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code9-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code9-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code9-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida9.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve percorrer o arranjo bidimensional e atribuir um valor para cada elemento da matriz, sendo a parte inteira do valor = ao número da linha e a parte fracionária = ao número da coluna.
    - Criar um arranjo bidimensional B, de números inteiros e de mesma dimensão de A, seguindo a lógica descrita em um pseudocódigo no roteiro, imprimindo esse arranjo depois.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de uma breve descrição do que o programa faz. Vamos analisar se o aluno preencheu esses campos corretamente.

- **Nome do aluno:** Marta Ferreira
- **Matrícula:** 752489
- **Data:** 29/10/2025
- **Descrição:** "Esse programa cria duas matrizes A e B de dimensão N completando de acordo com o pseudocódigo"

**Observação:** O aluno preencheu todos os campos solicitados, incluindo a descrição do programa.

#### Boas Práticas

O pylint apontou alguns problemas no código do aluno. Vamos analisar esses problemas, ignorando aqueles relacionados a docstrings, espaços em branco inúteis e nome do módulo.

- **Trailing whitespace (linhas 28 e 32):** Existem espaços em branco desnecessários no final dessas linhas. É importante manter o código limpo e sem espaços desnecessários no final das linhas.

**Sugestão:** Remova os espaços em branco

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code9-2.py", "Solucao/solucao-code9.py", "lint/code9-2-lint.pre", "ast/code9-2-ast.pre", "Esqueleto/esqueleto-code9.py", "Saida/saida9.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code9.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code9.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p10.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code9.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code9-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code9-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code9-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida9.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve percorrer o arranjo bidimensional e atribuir um valor para cada elemento da matriz, sendo a parte inteira do valor = ao número da linha e a parte fracionária = ao número da coluna.
    - Criar um arranjo bidimensional B, de números inteiros e de mesma dimensão de A, seguindo a lógica descrita em um pseudocódigo no roteiro, imprimindo esse arranjo depois.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno preenchesse o nome, matrícula e data, além de uma breve descrição do que o programa faz. Vamos analisar se esses campos foram preenchidos corretamente:

- **Nome do aluno:** Carla Maria
- **Matrícula:** 235748
- **Data:** 30/10/25
- **Descrição:** São duas matrizes quadradas: a primeira com valores da linha e coluna e a segunda com valores definidos pela diagonal principal.

**Observações:**
- O aluno preencheu todos os campos solicitados (nome, matrícula e data) corretamente.
- A descrição do programa está adequada e descreve claramente o que o programa faz.

#### Boas Práticas

O pylint apontou algumas observações sobre o código do aluno. Vamos analisar essas observações:

- **Trailing whitespace (espaços em branco desnecessários):**
  - Linha 24: Há um espaço em branco após o último caractere da linha.
  - Linha 28: Há um espaço em branco após o último caractere da 

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code9-3.py", "Solucao/solucao-code9.py", "lint/code9-3-lint.pre", "ast/code9-3-ast.pre", "Esqueleto/esqueleto-code9.py", "Saida/saida9.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code9.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code9.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p10.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code9.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code9-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code9-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code9-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida9.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve percorrer o arranjo bidimensional e atribuir um valor para cada elemento da matriz, sendo a parte inteira do valor = ao número da linha e a parte fracionária = ao número da coluna.
    - Criar um arranjo bidimensional B, de números inteiros e de mesma dimensão de A, seguindo a lógica descrita em um pseudocódigo no roteiro, imprimindo esse arranjo depois.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Análise do Código do Aluno

#### Descrição

No código base fornecido pelo professor, havia campos para que o aluno pudesse preencher com seu nome, matrícula e data, além de uma breve descrição do que o programa faz. Vamos analisar se esses campos foram preenchidos corretamente:

- **Nome do aluno:** O aluno preencheu com "Sabrina Guedes". Isso está correto.
- **Matrícula:** O aluno preencheu com "987456". Isso está correto.
- **Data:** O aluno preencheu com "25/10/2025". Isso está correto.
- **Descrição do programa:** O aluno deixou o campo como "Substitua esta linha por um breve comentáro dizendo o que o programa faz". Isso está incorreto, pois o aluno deveria ter substituído essa linha por uma breve descrição do que o programa faz.

**Feedback:** O aluno deve substituir o comentário padrão por uma breve descrição do programa, explicando que ele cria duas matrizes quadradas, preenche a matriz A com valores decimais baseados na linha e coluna, e a matriz B com valores inteiros segu